In [1]:
# import libraries
import numpy as np 
import pandas as pd 

import warnings 
warnings.filterwarnings("ignore")

from autoviz import AutoViz_Class

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from feature_engine.encoding import RareLabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import re

pd.set_option('display.max_rows', 1000)

Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


In [2]:
df = pd.read_parquet('./Flight_Delay.parquet')

In [3]:
length = df.shape[0]
print(f"Numner of data: {length}")
df.info()

Numner of data: 30132672
<class 'pandas.core.frame.DataFrame'>
Index: 30132672 entries, 1 to 596675
Data columns (total 29 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Year                       int64  
 1   Month                      int64  
 2   DayofMonth                 int64  
 3   FlightDate                 object 
 4   Marketing_Airline_Network  object 
 5   OriginCityName             object 
 6   DestCityName               object 
 7   CRSDepTime                 int64  
 8   DepTime                    float64
 9   DepDelay                   float64
 10  DepDelayMinutes            float64
 11  TaxiOut                    float64
 12  WheelsOff                  float64
 13  WheelsOn                   float64
 14  TaxiIn                     float64
 15  CRSArrTime                 int64  
 16  ArrTime                    float64
 17  ArrDelay                   float64
 18  ArrDelayMinutes            float64
 19  CRSElapsedTime        

In [4]:
# df_tail = df.tail(10000)

In [5]:
# df_tail.info()

In [6]:
# import gc
# # del df
# gc.collect()
# df = df_tail

In [7]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'FlightDate',
       'Marketing_Airline_Network', 'OriginCityName', 'DestCityName',
       'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'TaxiOut',
       'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay',
       'ArrDelayMinutes', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime',
       'Distance', 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [8]:
df['Marketing_Airline_Network'].value_counts()

Marketing_Airline_Network
AA    7716510
DL    6674517
UA    5657685
WN    5210031
AS    1598109
B6    1098362
NK     848118
F9     575147
G4     449977
HA     289961
VX      14255
Name: count, dtype: int64

In [9]:
df['OriginCityName'].value_counts()

OriginCityName
Chicago, IL                           1708957
Atlanta, GA                           1477845
Dallas/Fort Worth, TX                 1166054
Denver, CO                            1131732
New York, NY                          1106837
Charlotte, NC                          966926
Houston, TX                            894020
Los Angeles, CA                        877509
Washington, DC                         875581
Seattle, WA                            731924
Phoenix, AZ                            723473
Las Vegas, NV                          664066
Detroit, MI                            649629
Minneapolis, MN                        601369
San Francisco, CA                      600482
Orlando, FL                            581866
Newark, NJ                             576284
Boston, MA                             548144
Philadelphia, PA                       537146
Salt Lake City, UT                     492913
Baltimore, MD                          386100
Fort Lauderdale, FL

In [10]:
df['DestCityName'].value_counts()

DestCityName
Chicago, IL                           1731182
Atlanta, GA                           1517540
Dallas/Fort Worth, TX                 1179300
Denver, CO                            1144296
New York, NY                          1091655
Charlotte, NC                          987674
Houston, TX                            896815
Los Angeles, CA                        876185
Washington, DC                         871565
Seattle, WA                            733457
Phoenix, AZ                            729253
Las Vegas, NV                          670113
Detroit, MI                            651147
San Francisco, CA                      606657
Minneapolis, MN                        600162
Orlando, FL                            583006
Newark, NJ                             576539
Boston, MA                             548524
Philadelphia, PA                       537673
Salt Lake City, UT                     495349
Baltimore, MD                          390505
Miami, FL            

In [11]:
# Select useful columns
df['FlightDelay'] = df['ArrDelay'] - df['DepDelay']
selected_cols = ['FlightDelay', 'DepDelay', 'FlightDate', 'DepTime', 'Marketing_Airline_Network', 'OriginCityName', 'DestCityName', 'Distance']
df = df[selected_cols]
df.sample(5).T

,248646,133041,34213,388513,73192
FlightDelay,-6.0,-14.0,-4.0,-4.0,-24.0
DepDelay,-2.0,-3.0,1.0,67.0,-4.0
FlightDate,2022-09-11,2020-12-29,2018-03-23,2018-02-16,2020-05-29
DepTime,1743.0,932.0,1223.0,1937.0,756.0
Marketing_Airline_Network,AS,UA,AA,WN,UA
OriginCityName,"San Diego, CA","Houston, TX","Charleston, SC","Chicago, IL","Denver, CO"
DestCityName,"Seattle, WA","Fort Myers, FL","Washington, DC","Houston, TX","San Francisco, CA"
Distance,1050.0,861.0,444.0,937.0,967.0


In [12]:
# The label to predict
main_label = 'FlightDelay'
# convert date to datetime and extract features
for col in ['FlightDate']:
    df[col] = pd.to_datetime(df[col], format="%Y-%m-%d")
# df['Year'] = df['FlightDate'].dt.year
# df['Month'] = df['FlightDate'].dt.month_name()
df['Day'] = df['FlightDate'].dt.day_name()

In [13]:
# Extract hours and minutes from DepTime
df['DepHour'] = df['DepTime'] // 100
df['DepMinute'] = df['DepTime'] % 100

# More precise time representation (hour + fraction of the hour)
df['DepFraction'] = df['DepHour'] + df['DepMinute'] / 60
df['DepTime_sin'] = np.sin(2 * np.pi * df['DepFraction'] / 24)
df['DepTime_cos'] = np.cos(2 * np.pi * df['DepFraction'] / 24)

In [14]:
# drop unused columns
cols2drop = ['FlightDate', 'DepTime']
df = df.drop(cols2drop, axis=1)

In [15]:
df.sample(5).T

,155752,579941,408899,153183,349721
FlightDelay,-13.0,-10.0,-13.0,-8.0,-16.0
DepDelay,-9.0,-7.0,-10.0,107.0,12.0
Marketing_Airline_Network,AA,AA,AA,DL,AA
OriginCityName,"Stillwater, OK","Phoenix, AZ","Dallas/Fort Worth, TX","New York, NY","Washington, DC"
DestCityName,"Dallas/Fort Worth, TX","Baltimore, MD","Manhattan/Ft. Riley, KS","Burlington, VT","Charleston, SC"
Distance,225.0,1999.0,431.0,258.0,444.0
Year,2023,2018,2020,2022,2019
Month,February,August,March,September,April
Day,Sunday,Sunday,Tuesday,Friday,Monday
DepHour,14.0,15.0,8.0,23.0,12.0


In [16]:
# bin numerical cols in equal qcuts
def bin_column(df, col_name, num_bins=12):

    # Calculate the bin edges to evenly split the numerical column
    bin_edges = pd.qcut(df[col_name], q=num_bins, retbins=True)[1]

    # Define labels for the categorical bins based on bin edges
    bin_labels = [f'{int(bin_edges[i])}_{int(bin_edges[i+1])}' for i in range(num_bins)]

    # Use pd.qcut to create quantile-based bins with equal number of records in each bin
    df[col_name] = pd.qcut(df[col_name], q=num_bins, labels=False)

    # Update the bin labels to be more descriptive
    df[col_name] = df[col_name].map(lambda x: bin_labels[x])
    
    # Convert the column to object dtype
    df[col_name] = df[col_name].fillna('None').astype('object')

    return df

In [17]:
for col in ['Distance', 'DepDelay', 'FlightDelay']:
    print(col)
    df = bin_column(df, col)
df.sample(5).T

Distance
DepDelay
FlightDelay


,142076,152555,78467,459877,57142
FlightDelay,-10_-8,-1274_-21,1_7,-4_-2,1_7
DepDelay,0_3,-7_-6,-10_-7,17_52,-5_-4
Marketing_Airline_Network,WN,DL,AA,WN,AS
OriginCityName,"Los Angeles, CA","New York, NY","Dallas/Fort Worth, TX","Las Vegas, NV","Seattle, WA"
DestCityName,"Phoenix, AZ","Miami, FL","Salt Lake City, UT","Phoenix, AZ","Medford, OR"
Distance,352_430,1011_1205,871_1011,197_279,279_352
Year,2018,2022,2018,2018,2021
Month,November,December,August,April,April
Day,Tuesday,Wednesday,Friday,Monday,Thursday
DepHour,19.0,7.0,10.0,22.0,15.0


In [18]:
for col in ['Marketing_Airline_Network', 'OriginCityName', 'DestCityName']:
    print(col)
    df[col] = df[col].fillna('None')
    encoder = RareLabelEncoder(n_categories=1, max_n_categories=200, replace_with='Other', tol=0.0001)
    df[col] = encoder.fit_transform(df[[col]])

Marketing_Airline_Network
OriginCityName
DestCityName


In [19]:
df['Marketing_Airline_Network'].value_counts()

Marketing_Airline_Network
AA    7716510
DL    6674517
UA    5657685
WN    5210031
AS    1598109
B6    1098362
NK     848118
F9     575147
G4     449977
HA     289961
VX      14255
Name: count, dtype: int64

In [20]:
df['OriginCityName'].value_counts()

OriginCityName
Chicago, IL                           1708957
Atlanta, GA                           1477845
Dallas/Fort Worth, TX                 1166054
Denver, CO                            1131732
New York, NY                          1106837
Charlotte, NC                          966926
Houston, TX                            894020
Los Angeles, CA                        877509
Washington, DC                         875581
Seattle, WA                            731924
Phoenix, AZ                            723473
Other                                  694198
Las Vegas, NV                          664066
Detroit, MI                            649629
Minneapolis, MN                        601369
San Francisco, CA                      600482
Orlando, FL                            581866
Newark, NJ                             576284
Boston, MA                             548144
Philadelphia, PA                       537146
Salt Lake City, UT                     492913
Baltimore, MD      

In [21]:
df['DestCityName'].value_counts()

DestCityName
Chicago, IL                           1731182
Atlanta, GA                           1517540
Dallas/Fort Worth, TX                 1179300
Denver, CO                            1144296
New York, NY                          1091655
Charlotte, NC                          987674
Houston, TX                            896815
Los Angeles, CA                        876185
Washington, DC                         871565
Seattle, WA                            733457
Phoenix, AZ                            729253
Other                                  688451
Las Vegas, NV                          670113
Detroit, MI                            651147
San Francisco, CA                      606657
Minneapolis, MN                        600162
Orlando, FL                            583006
Newark, NJ                             576539
Boston, MA                             548524
Philadelphia, PA                       537673
Salt Lake City, UT                     495349
Baltimore, MD        

In [22]:
counts = df['OriginCityName'].value_counts()
other_count = counts.get('Other', 0) 
print(f"Number of 'Other': {other_count}")
unique_count = df['OriginCityName'].nunique()
print(f"Number of unique values: {unique_count}")

Number of 'Other': 694198
Number of unique values: 201


In [23]:
counts = df['DestCityName'].value_counts()
other_count = counts.get('Other', 0) 
print(f"Number of 'Other': {other_count}")
unique_count = df['DestCityName'].nunique()
print(f"Number of unique values: {unique_count}")

Number of 'Other': 688451
Number of unique values: 201


In [24]:
for col in ['Distance', 'DepDelay', 'FlightDelay']:
    print(df[col].value_counts())

Distance
197_279      2530661
430_529      2529864
16_197       2518755
871_1011     2517047
620_735      2515966
279_352      2513636
735_871      2511397
1698_5812    2508346
1205_1698    2507480
1011_1205    2502943
352_430      2497462
529_620      2479115
Name: count, dtype: int64
DepDelay
-10_-7      4191969
-2_0        3068778
-6_-5       2633538
-342_-10    2534169
17_52       2517813
3_17        2496345
52_7223     2483218
-5_-4       2417812
-4_-3       2262998
-7_-6       2110097
-3_-2       1980911
0_3         1435024
Name: count, dtype: int64
FlightDelay
-1274_-21    2923417
-17_-14      2805914
-10_-8       2657496
-8_-6        2599083
-12_-10      2555251
-21_-17      2497417
7_1448       2476434
-2_1         2465988
-6_-4        2402295
1_7          2337805
-14_-12      2307548
-4_-2        2104024
Name: count, dtype: int64
